# Day 3c

So far, we have treated the language model as a fixed feature extractor:
the model produces embeddings, and all task-specific training happens in a separate, linear classifier (`RidgeClassifierCV`).

Now we allow the language model itself to **adapt to the task**.

A straightforward way to do this would be full fine-tuning, where all model parameters are tuned to the task.

However, full fine-tuning: (1) updates millions of parameters, (2) requires substantial GPU memory and time, (3) and is often unnecessary for relatively small classification tasks.

Instead, we use [**LoRA (Low-Rank Adaptation)**](https://arxiv.org/abs/2106.09685), a parameter-efficient fine-tuning method.

LoRA inserts small, trainable low-rank matrices into the model’s attention layers, keeps the original pretrained weights frozen, and updates less than 1% of the total parameters.

This keeps training fast and lightweight, while still allowing task-specific learning.

**Credits**: We would like to thank [Taisiia Tikhomirova](https://www.mpib-berlin.mpg.de/staff/taisiia-tikhomirova) and [Valentin Kriegmair](https://www.mpib-berlin.mpg.de/person/valentin-kriegmair/171978) for creating this exercise.

## Environment Setup
Make sure to set your runtime to use a GPU by going to `Runtime` -> `Change runtime type` -> `Hardware accelerator` -> `T4 GPU`

In [ ]:
import sys
if 'google.colab' in sys.modules:
    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')

    !pip -q install transformers datasets evaluate accelerate peft

    # Change working directory
    %cd /content/drive/MyDrive/LLM4BeSci_Ljubljana2026/day_3

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, TaskType
import evaluate

## Load the same data and prepare labels


We reuse:
- `media_bias_train.csv`
- `media_bias_test.csv`

Columns:
- `text`: tweet text
- `bias`: label


Transformers expect integer labels. We create a mapping from label name → integer ID using the training set.

In [ ]:
# 1. Load Data
media_bias_train = pd.read_csv("media_bias_train.csv")
media_bias_test  = pd.read_csv("media_bias_test.csv")

media_bias_train

We create a dictionary to map string labels to integers and vice versa.

In [ ]:
# 2. Prepare Label Mappings
label_names = sorted(media_bias_train["bias"].unique())
label2id = {name: i for i, name in enumerate(label_names)}
id2label = {i: name for name, i in label2id.items()}

print(f"label2id: {label2id}\nid2label: {id2label}")

We create a new column 'label' which the Trainer (see below) specifically looks for by default.

In [ ]:
# 3. Apply mapping to DataFrames
media_bias_train["label"] = media_bias_train["bias"].map(label2id)
media_bias_test["label"]  = media_bias_test["bias"].map(label2id)

# Quick check
print(label2id)
media_bias_train.head(3)

## Tokenization and dataset conversion
The code next loads the right tokenizer for the model (`"all-MiniLM-L6-v2"`), converts the `pd.DataFrame`s to Hugging Face Datasets, and tokenizes the texts.

In [ ]:
# Define the base model - MiniLM (a small BERT-based model) to fit in memory/compute constraints
model_ckpt = "sentence-transformers/all-MiniLM-L6-v2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# Tokenizes the text. Truncation=True ensures texts longer than max_length are cut off.
def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, max_length=256, padding = "max_length")

# Convert Pandas DataFrames to Hugging Face Datasets
train_ds = Dataset.from_pandas(media_bias_train[["text", "label"]])
test_ds  = Dataset.from_pandas(media_bias_test[["text", "label"]])

# Apply tokenization
# We remove the 'text' column because the model only needs the numerical 'input_ids'
train_ds = train_ds.map(tokenize, batched=True).remove_columns(["text"])
test_ds  = test_ds.map(tokenize, batched=True).remove_columns(["text"])

## LoRA setup (PEFT)

The code next loads the model (`"all-MiniLM-L6-v2"`) and applies LoRA adapters to the attention projections.

For `"all-MiniLM-L6-v2"` (which is BERT-based), common LoRA targets are `query` and `value`.
This updates only a small number of parameters while leaving the base model frozen.


**LoRA parameters**

- `r`: the rank of the low-rank adapters.  
  Higher values give the model more capacity to adapt, but add more trainable parameters.

- `lora_alpha`: a scaling factor for the LoRA updates.  
  It controls how strongly the adapters influence the original model weights.

- `lora_dropout`: dropout applied inside the LoRA adapters during training.  
  This helps regularization and can reduce overfitting.

In [ ]:
# Load the base model
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    device_map="cuda"
)

# LoRA Configuration
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, # Sequence Classification
    r=8,                        # Rank: The dimension of the low-rank matrices. Higher = more parameters.
    lora_alpha=16,              # Alpha: Scaling factor. Usually set to 2x rank. Controls weight of adapter.
    lora_dropout=0.05,          # Dropout probability for LoRA layers
    target_modules=["query", "value"], # Modules to apply LoRA to. For MiniLM/BERT, query/value is standard.
)

# Wrap the base model with the LoRA configuration
model = get_peft_model(model, lora_config)

# Verify trainable parameters
# You should see a very low percentage (usually <1%)
model.print_trainable_parameters()

## Training

We train with HuggingFace `Trainer`.

**Key training parameters**

- `num_train_epochs`: how many times the model sees the full training dataset.  More epochs allow better learning but may lead to overfitting.

- `learning_rate`: size of each update step during training.  Smaller values are more stable; larger values learn faster but can be unstable.

- `logging_steps`: how often training progress is printed.

- `save_strategy`: controls whether model checkpoints are saved during training.  Here we disable saving.

- `report_to`: enables/disables external logging tools (e.g., Weights & Biases).

In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./lora_miniLM_b",
    per_device_train_batch_size=64,
    logging_steps = 50,
    num_train_epochs=15, # will take a bit of time
    learning_rate=2e-4, # LoRA usually requires a higher LR
    save_strategy="no",
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model,  # LoRA-augmented MiniLM model whose trainable parameters will be optimized
    args=training_args,  # Training hyperparameters (learning rate, epochs, batch size, logging, etc.)
    train_dataset=train_ds,  # Tokenized training data providing inputs and labels
    data_collator=DataCollatorWithPadding(tokenizer),  # Pads sequences dynamically per batch using the tokenizer
)

# Start Training
trainer.train()

It is important to note that the training loss printed here is the average **cross entropy loss** per sample (not the accuracy!).

### Evaluation

Instead of writing our own evaluation loop, we use HuggingFace’s built-in **Trainer.evaluate()** method.

In [ ]:
# Load a standard accuracy metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """
    This function tells the Trainer how to measure performance.

    It receives:
    - model outputs (logits)
    - the correct labels

    It returns:
    - classification accuracy
    """
    logits, labels = eval_pred

    # Convert model scores into predicted class labels
    preds = logits.argmax(axis=-1)

    # Compare predictions to true labels and compute accuracy
    return accuracy_metric.compute(
        predictions=preds,
        references=labels
    )

# Create a Trainer for the LoRA-fine-tuned model
lora_trainer = Trainer(
    model=trainer.model,     # model after LoRA fine-tuning
    args=training_args,      # evaluation settings
    eval_dataset=test_ds,   # test data
    tokenizer=tokenizer,     # tokenizer
    compute_metrics=compute_metrics,  # accuracy computation
)

# Run evaluation
lora_metrics = lora_trainer.evaluate()

# Print accuracy after fine-tuning
print("Accuracy after fine-tuning:", lora_metrics["eval_accuracy"])

**TASK 1:** LoRA sweep (capacity vs performance)
Try:
- `r ∈ {4, 8, 16}`
- `lora_alpha ∈ {8, 16, 32}`

How does the performance change? Why do you think it is?